[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vanguardfox/ads509-group5/blob/main/ADS509_Final_Project.ipynb)

# Evaluating Financial Independence / Retire Early (FIRE) Thresholds by Demographic

What is the Financial Independence number the FIRE community is aiming for by age, location, and over time?

# Group 5 Objective:
 The purpose of this project is to evaluate the FIRE financial thresholds by demographic groups such as age and location as well as changes in trends over times. Resulting analysis can be used in a variety of applications including leading economic sentiment predictions, generational assessment of financial health, and others via a Flask implementation.

## Reddit Scrape

Community: r/FIRE \
thread: /What_is_your_fire_number/ \
Code Reference: with help from ChatGPT https://chatgpt.com/ and https://colab.research.google.com/


**Privacy note:**

*   Credentials were created for the class but might have to figure out how to hide
*   Username will also have to be masked to be anonymous

(Optional: Install packages as needed)

In [ ]:
#!pip install requests beautifulsoup4

## Import Libraries

In [ ]:
# Library
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.auth import HTTPBasicAuth
import json
import csv
from os import read

## Initialize API Credentials

In [ ]:
# Reddit API credentials
client_id = '8-xAlbBmc1_FYraSwJENtw'
client_secret = 'Jg9JyK9R_ixyNESMaGE3BwLzmBhf2w'
username = 'Western_Signal4499'
password = 'ADS509finAl'
user_agent = 'ADS509Scrapper/0.1 by Western_Signal4499'

# Get the OAuth token
auth = HTTPBasicAuth(client_id, client_secret)
headers = {'User-Agent': user_agent}
data = {'grant_type': 'password', 'username': username, 'password': password}

response = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, headers=headers, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
else:
    print(f"Error: {response.status_code}, {response.text}")

## Use access_token field from JSON to query threads

In [ ]:
# Use the token to search for threads
headers['Authorization'] = f'bearer {token}'

# Constructs a string for the r/FIRE URL to query "what is your fire number"
# restrict_sr=1 ensures that the query is constrained to only this subreddit
search_url = "https://oauth.reddit.com/r/FIRE/search?q=what%20is%20your%20fire%20number&restrict_sr=1"

# Store results of GET requrest to the API for the given "search_url"
search_response = requests.get(search_url, headers=headers)

# If the API GET is successful...
if search_response.status_code == 200: # Code 200 means the request was flagged as successful
    search_data = search_response.json() # Parse the JSON data from the response
    #print(search_data)
    threads = search_data['data']['children'] # Extract the list of threads

    # Initialize a CSV and writes an head with columns (within .writerow() arg)
    with open('fire_comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Thread ID', 'Title', 'User', 'Comment'])

        # Iterate through each thread from API's response JSON
        for thread in threads:
            thread_title = thread['data']['title'] # Parse and extract title
            thread_id = thread['data']['id'] # Parse and extract id
            # Prepare URL as a string for better code handling
            comments_url = f"https://oauth.reddit.com/r/FIRE/comments/{thread_id}.json"

            # Store results of GET to fetch comments of the current iterated thread
            comments_response = requests.get(comments_url, headers=headers)
            if comments_response.status_code == 200: # Again, check if GET is successful
                comments_data = comments_response.json() # Store JSON
                comments = comments_data[1]['data']['children'] # Parse and extract comments

                # Iterate through each comment
                for comment in comments:
                    if comment['kind'] == 't1': # Only iterate on top comments
                        username = comment['data']['author'] # Extract username
                        body = comment['data']['body'] # Extract text body
                        # Write parsed data into respective CSV row, mapped to column
                        writer.writerow([thread_id, thread_title, username, body])

                print(f"Thread ID: {thread_id}, Title: {thread_title}, User1: {username}, Comment: {body}")

            else:
                print(f"Failed to retrieve comments for thread ID {thread_id} with status code {comments_response.status_code}")

else:
    print(f"Failed to retrieve search results with status code {search_response.status_code}")

Thread ID: 1fqzb0p, Title: What is your fire number? , User1: muy_carona, Comment: $8,675,309
Thread ID: rfxxkz, Title: What is your fire number?, User1: xmxprztm, Comment: 4M at least. Going to move to Spain
Thread ID: zddhma, Title: For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?, User1: [deleted], Comment: Some services or experiences are only available in hcol areas. Good fine dining options and/or weather and or ___.
Thread ID: 10ex17r, Title: What is your FIRE number?, User1: zenjdg, Comment: My ideal scenario is $100k per month in tax-free cash flow.
Thread ID: z5a29v, Title: What is your fire number and how did you arrive at it?, User1: [deleted], Comment: [deleted]
Thread ID: 1ef8jjf, Title: How to split finances when one is FIRE'd and one isn't FIRE'ing, User1: [deleted], Comment: Break up
Thread ID: oh6i17, Title: What is your FIRE number and at what age do you want to achieve it by?, User1: FIREDOC627, Comment: W

## Verify the contents of the CSV that was created above and store as DataFrame

In [ ]:
rfire_df = pd.read_csv('fire_comments.csv')
rfire_df.head()

,Thread ID,Title,User,Comment
0,1fqzb0p,What is your fire number?,manimopo,1.5m with paid off houses
1,1fqzb0p,What is your fire number?,Animag771,I'm just waiting for someone to say something ...
2,1fqzb0p,What is your fire number?,wowsocool4u,$1.9m. Targeting a few months after i turn 50...
3,1fqzb0p,What is your fire number?,Aromatic_Heart,"On days I hate my job, 600k. The other days, d..."
4,1fqzb0p,What is your fire number?,Ziqach,$2M with a paid off house. So stressful to see...


## Explore essential data on the dataframe

In [ ]:
rfire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Thread ID  1462 non-null   object
 1   Title      1462 non-null   object
 2   User       1462 non-null   object
 3   Comment    1462 non-null   object
dtypes: object(4)
memory usage: 45.8+ KB


## Obtain unique values in a column (in this instance, Title)

In [ ]:
rfire_df['Title'].unique()

array(['What is your fire number? ', 'What is your fire number?',
       'For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?',
       'What is your FIRE number?',
       'What is your fire number and how did you arrive at it?',
       "How to split finances when one is FIRE'd and one isn't FIRE'ing",
       'What is your FIRE number and at what age do you want to achieve it by?',
       'We’re all privileged',
       'How my wife and I will retire before 50; neither of us make over $75K',
       '10 tips from my 20 year journey to financial freedom',
       'Is toxic corporate culture why most of us want to Fire? ',
       'The cliff is steep: How do you choose to retire early when working 5 more years can bring so much more',
       'How did you come up with your FIRE number? What is the best calculator, way to get at it?',
       'Forgive me if this is too personal, but what was your estimated net worth around 18 and how did 

# Flask Environment

Import Packages

In [6]:
# Import packages
#!pip install Flask
#!pip install pyngrok

## Upload Libraries

In [1]:
# Libraries:
from flask import Flask, request, jsonify
import threading
from pyngrok import ngrok

## Simple Flask Application

In [2]:
# Set up ngrok authtoken
!ngrok authtoken 2mxORvZevUlWiZcP42H0kKWt5Db_62uzFBWQsdosv7Eau5TWZ

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the FIRE Number Analyzer!"

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    # Placeholder for analysis logic
    return jsonify({"message": "Analysis complete!"})

# Function to run the Flask app
def run_app():
    app.run(port=5000)

# Start Flask app in a new thread
threading.Thread(target=run_app).start()

# Set up a tunnel to the Flask app
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "NgrokTunnel: "https://6fb1-35-236-152-11.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
